<a href="https://colab.research.google.com/github/G0nkly/pytorch_sandbox/blob/main/vit/VIT_First_try.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import einops
from tqdm.notebook import tqdm
from torchsummary import summary

import torch
from torch import nn
import torchvision
import torch.optim as optim
from torchvision.transforms import Compose, Resize, ToTensor, Normalize, RandomHorizontalFlip, RandomCrop

In [2]:
# https://www.youtube.com/watch?v=nZ22Ecg9XCQ

In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [4]:
###################
# HYPERPARAMETERS #
###################

In [5]:
patch_size = 16
latent_size = 768
n_channels = 3
num_heads = 12
num_encoders = 12
dropout = 0.1
num_classes = 10
size = 224

epochs = 10
base_lr = 10e-3
weight_decay = 0.03
batch_size = 4

In [6]:
###########################
# INPUT LINEAR PROJECTION #
###########################

In [9]:
class InputEmbedding(nn.Module):

  def __init__(self, patch_size=patch_size, n_channels=n_channels, device=device, latent_size=latent_size, batch_size=batch_size):
    super(InputEmbedding, self).__init__()
    self.latent_size = latent_size
    self.patch_size = patch_size
    self.n_channels = n_channels
    self.device = device
    self.batch_size = batch_size
    self.input_size = self.patch_size * self.patch_size * self.n_channels
    # linear projection
    self.linear_projection = nn.Linear(self.input_size, self.latent_size)
    # class token
    self.class_token = nn.Parameter(torch.randn(self.batch_size, 1, self.latent_size)).to(self.device)
    # positional embedding
    self.pos_embedding = nn.Parameter(torch.randn(self.batch_size, 1, self.latent_size)).to(self.device)

  def forward(self, input_data):
    input_data = input_data.to(self.device)

    # Patchify input image
    patches = einops.rearrange(
        input_data, 'b c (h h1) (w w1) -> b (h w) (h1 w1 c)', h1=self.patch_size, w1=self.patch_size
    )

    print(input_data.size())
    print(patches.size())

    linear_projection = self.linear_projection(patches).to(device)
    b, n, _ = linear_projection.shape

    linear_projection = torch.cat((self.class_token, linear_projection), dim=1)

In [8]:
test_input = torch.randn((1, 3, 224, 224))
test_class = InputEmbedding().to(device)
test_class(test_input)

torch.Size([1, 3, 224, 224])
torch.Size([1, 196, 768])
